In [3]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [4]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython
model = MatrixFactorization_BPR_Cython

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

EvaluatorHoldout: Ignoring 2620 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [5]:
import optuna as op
def objective(trial):
    # Optuna suggest params
    sgd_mode = trial.suggest_categorical("sgd_mode", ["sgd", "adagrad", "adam"])
    epochs = 1500
    num_factors = trial.suggest_int("num_factors", 10, 300)
    batch_size = trial.suggest_categorical("batch_size", [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    positive_reg = trial.suggest_loguniform("positive_reg", 1e-6, 1e-1)
    negative_reg = trial.suggest_loguniform("negative_reg", 1e-6, 1e-1)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
    random_seed = 1234
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [8]:
best_params = {
    "sgd_mode": "adam",
    "epochs": 240,
    "num_factors": 148,
    "batch_size": 16,
    "positive_reg": 0.000439293634625917,
    "negative_reg": 5.495056923670675e-05,
    "learning_rate": 0.0007794513725937752,
    "random_seed": 1234
}

# {"sgd_mode": "adam", "epochs": 240, "num_factors": 148, "batch_size": 16, "positive_reg": 0.000439293634625917, "negative_reg": 5.495056923670675e-05, "learning_rate": 0.0007794513725937752}
study_name = "MFACT_BPR-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=30)

[I 2023-11-23 13:05:28,129] Using an existing study with name 'MFACT_BPR-study' instead of creating a new one.
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  positive_reg = trial.suggest_loguniform("positive_reg", 1e-6, 1e-1)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  negative_reg = trial.suggest_loguniform("negative_reg", 1e-6, 1e-1)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v

MatrixFactorization_BPR_Cython_Recommender: URM Detected 467 ( 3.7%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 330 ( 1.5%) items with no interactions.
MF_BPR: Processed 12640 (100.0%) in 0.39 sec. MSE loss 2.89E-02. Sample per second: 32809
MF_BPR: Epoch 1 of 300. Elapsed time 0.12 sec
MF_BPR: Processed 12640 (100.0%) in 0.50 sec. MSE loss 3.11E-02. Sample per second: 25358
MF_BPR: Epoch 2 of 300. Elapsed time 0.23 sec
MF_BPR: Processed 12640 (100.0%) in 0.61 sec. MSE loss 3.50E-02. Sample per second: 20638
MF_BPR: Epoch 3 of 300. Elapsed time 0.35 sec
MF_BPR: Processed 12640 (100.0%) in 0.73 sec. MSE loss 4.16E-02. Sample per second: 17401
MF_BPR: Epoch 4 of 300. Elapsed time 0.46 sec
MF_BPR: Processed 12640 (100.0%) in 0.84 sec. MSE loss 5.09E-02. Sample per second: 15096
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 10018 (100.0%) in 4.42 sec. Users per second: 2264
MF_BPR: CUTOFF: 10 - PRECISION: 0.0004093, PRECISION_RECALL_MIN_

[I 2023-11-23 13:10:15,130] Trial 3 finished with value: 0.008225539267413895 and parameters: {'sgd_mode': 'adam', 'num_factors': 148, 'batch_size': 16, 'positive_reg': 0.000439293634625917, 'negative_reg': 5.495056923670675e-05, 'learning_rate': 0.0007794513725937752}. Best is trial 0 with value: 0.00866921205642897.


MatrixFactorization_BPR_Cython_Recommender: URM Detected 467 ( 3.7%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 330 ( 1.5%) items with no interactions.


/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  positive_reg = trial.suggest_loguniform("positive_reg", 1e-6, 1e-1)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  negative_reg = trial.suggest_loguniform("negative_reg", 1e-6, 1e-1)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  le

...

[I 2023-11-23 15:49:38,342] Trial 31 finished with value: 0.0003517002728422173 and parameters: {'sgd_mode': 'adagrad', 'num_factors': 224, 'batch_size': 16, 'positive_reg': 0.0015533757378042876, 'negative_reg': 0.00014245248065974955, 'learning_rate': 0.0027304697438653063}. Best is trial 7 with value: 0.008853955736816549.


MatrixFactorization_BPR_Cython_Recommender: URM Detected 467 ( 3.7%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 330 ( 1.5%) items with no interactions.


/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  positive_reg = trial.suggest_loguniform("positive_reg", 1e-6, 1e-1)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  negative_reg = trial.suggest_loguniform("negative_reg", 1e-6, 1e-1)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_5608/686704151.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  le

MF_BPR: Processed 12640 (100.0%) in 0.68 sec. MSE loss 3.02E-02. Sample per second: 18457
MF_BPR: Epoch 1 of 300. Elapsed time 0.11 sec
MF_BPR: Processed 12640 (100.0%) in 0.80 sec. MSE loss 3.08E-02. Sample per second: 15716
MF_BPR: Epoch 2 of 300. Elapsed time 0.23 sec
MF_BPR: Processed 12640 (100.0%) in 0.92 sec. MSE loss 3.38E-02. Sample per second: 13671
MF_BPR: Epoch 3 of 300. Elapsed time 0.35 sec
MF_BPR: Processed 12640 (100.0%) in 1.05 sec. MSE loss 4.02E-02. Sample per second: 11986
MF_BPR: Epoch 4 of 300. Elapsed time 0.49 sec
MF_BPR: Processed 12640 (100.0%) in 0.19 sec. MSE loss 4.60E-02. Sample per second: 67412
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 10018 (100.0%) in 5.05 sec. Users per second: 1983
MF_BPR: CUTOFF: 10 - PRECISION: 0.0004093, PRECISION_RECALL_MIN_DEN: 0.0008049, RECALL: 0.0006991, MAP: 0.0001110, MAP_MIN_DEN: 0.0002373, MRR: 0.0011101, NDCG: 0.0005804, F1: 0.0005163, HIT_RATE: 0.0040926, ARHR_ALL_HITS: 0.0011101, NOVELTY: 0.0068288, AVER

[I 2023-11-23 15:55:12,743] Trial 32 finished with value: 0.008239743857881208 and parameters: {'sgd_mode': 'adam', 'num_factors': 148, 'batch_size': 16, 'positive_reg': 0.0005367956772539367, 'negative_reg': 4.534805575945766e-05, 'learning_rate': 0.0006834036712055008}. Best is trial 7 with value: 0.008853955736816549.


In [9]:
study.best_params
# epoch = 260

{'sgd_mode': 'adagrad',
 'num_factors': 219,
 'batch_size': 32,
 'positive_reg': 0.016211221706684466,
 'negative_reg': 0.00012672106970228173,
 'learning_rate': 0.08036400470449359}

In [10]:
final = model(URM_train_validation)
final.fit(**study.best_params, epochs=260, random_seed=1234)

MatrixFactorization_BPR_Cython_Recommender: URM Detected 237 ( 1.9%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 111 ( 0.5%) items with no interactions.
MF_BPR: Processed 12640 (100.0%) in 0.69 sec. MSE loss 5.71E-01. Sample per second: 18202
MF_BPR: Epoch 1 of 260. Elapsed time 0.11 sec
MF_BPR: Processed 12640 (100.0%) in 0.81 sec. MSE loss 1.47E+00. Sample per second: 15666
MF_BPR: Epoch 2 of 260. Elapsed time 0.22 sec
MF_BPR: Processed 12640 (100.0%) in 0.92 sec. MSE loss 2.17E+00. Sample per second: 13711
MF_BPR: Epoch 3 of 260. Elapsed time 0.34 sec
MF_BPR: Processed 12640 (100.0%) in 1.04 sec. MSE loss 2.66E+00. Sample per second: 12170
MF_BPR: Epoch 4 of 260. Elapsed time 0.46 sec
MF_BPR: Processed 12640 (100.0%) in 0.15 sec. MSE loss 3.14E+00. Sample per second: 83931
MF_BPR: Epoch 5 of 260. Elapsed time 0.57 sec
MF_BPR: Processed 12640 (100.0%) in 0.27 sec. MSE loss 3.65E+00. Sample per second: 47410
MF_BPR: Epoch 6 of 260. Elapsed time

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2218 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10420 (100.0%) in 5.95 sec. Users per second: 1750


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.031555                 0.061472  0.054007  0.012733    0.025207   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.098538  0.050914  0.039835  0.244626      0.111836  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.824498          0.201693    0.824498       0.054326   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10             9.14083                   0.992109             0.155907   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [ ]:
## MAP = .0127328